In [1]:
from Crypto.Util.number import getPrime, isPrime, long_to_bytes, inverse
from Crypto.Cipher import AES
from hashlib import sha256
from sympy import Matrix, gcd, divisors

In [2]:
n = 875160154252520186251141359813376844205078837688859219647998263059390545455666959868272831956779132045330105458640460325486680058028233270020291614157451317894110269639780507518860757299450578151147748572019371950843986400076289065334864577087504297359676338166424192877389197803442490978821375794048316553853085966558794240760959161987598108459441945291723309230665829568115073301763544532552179053221908864303864014278134933463597319119189753348991487712539547783209384907074459767339389912999640300731884080101606842315180302622347685382825488775275363493924203654141256657466019975768214244502429057343992946031175889606627829209702720334661107181627261133702840164098042293465302874342367
rns = [45948196110742333078791904670754464213578139076280109814760521831353273605782767324779081189039031009815013571451298379981707915542440414220083426012609548317078548002379773850309022768943249397294154504631709396600886029113258544, 665421018479354408650321321446504004021595291664664613059639369284378526681870680990726015080235767661978417143890017835302285399955943814186163819783105747360598346167878408804511855184450139017542601099343516459031570045750334635, 116901116055486713179543809270968616175254891027356317875260251653063459332706332539151662885597749087527500088196623717000607610204151297409257838680010556067036481294394312559141590536968964270065614095041734217016802912127551399, 257185387102952423422505329148493981831897715690584197952849048768011651303898015179171902245107615610642197589412557491017458513344338002354160349045327465820185865360211004133692933165886876481285477189851986291296987966454584913, 600597039831835988459465667623398229366883082253281715825043750022005882279772326647162809398726368298023180858058246314753665389921411839577046046481816334727242911521098595075675355340100111145925656700034968689114367721111389240]
e = 0x10001

In [3]:
M1 = Matrix([
    [rns[0], rns[1], 1],
    [rns[1], rns[2], 1],
    [rns[2], rns[3], 1]
])
r1 = M1.det()

M2 = Matrix([
    [rns[1], rns[2], 1],
    [rns[2], rns[3], 1],
    [rns[3], rns[4], 1]
])
r2 = M2.det()

We know that $r1 \equiv 0\,[r]$ and $r2 \equiv 0\,[r]$. See http://www.reteam.org/papers/e59.pdf.

In [4]:
for div in divisors(gcd(r1, r2)):
    if isPrime(div):
        r = div

In [5]:
r1 % r, r2 % r, r >> 767

(0, 0, 1)

Great, $r$ has $768$ bits.

Now solving a linear congruential system to find the parameters of the LCG. See e.g. https://research.nccgroup.com/wp-content/uploads/2020/07/randomness.pdf.

In [6]:
q = (rns[0]*rns[2] - rns[1]**2) * pow(rns[0] - rns[1], -1, r) % r
isPrime(q), q >> 767

(True, 1)

In [7]:
p = (rns[1] - q) * pow(rns[0], -1, r) % r
isPrime(p), p >> 767

(True, 1)

Now decrypting the ciphertext.

In [8]:
n = p * q * r
phi = (p - 1) * (q - 1) * (r - 1)
d = inverse(e, phi)

key = sha256(long_to_bytes(d)).digest()
cipher = AES.new(key, AES.MODE_ECB)
ct = "df534b412fbbb5bf920a9c8a76f85013b2c6cb49642c30a7e5e801f9576b4f36071945f46d3e78c8b1e2adff0c090d7d88e9a068caae958d87ca56bcd0763e00e527371f271530bd6bb779284233ec45"
pt = cipher.decrypt(bytes.fromhex(ct))
pt

b'HTB{0op5_my_$33d$_f311_0ff_5h3_gr0und_4nd_br0ugh5_y0u_4_fl4g!#@$%}\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e\x0e'